# Extracción de caracteristicas proyecto:
# Clasificación de supernovas de ALeRCE - ZTF
### Por Joaquin Zepeda y Benjamin Irarrazabal - Tutor: Pablo Montero


In [ ]:
# pyarrow might be needed to read the data
!python -m pip install Cython
!python -m pip install -e git+https://git@github.com/alercebroker/turbo-fats#egg=turbofats
!python -m pip install -e git+https://git@github.com/alercebroker/mhps#egg=mhps
!python -m pip install -e git+https://git@github.com/alercebroker/P4J#egg=P4J
!python -m pip install pyarrow
!python -m pip install -e git+https://git@github.com/alercebroker/lc_classifier#egg=lc_classifier

In [98]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
inicio_ejecucion = time.time()
# Se imprime la version de Tensorflow
print('Tensorflow version', tf.__version__)

Tensorflow version 2.4.0


In [84]:
from lc_classifier.features import MHPSExtractor, PeriodExtractor, GPDRWExtractor
from lc_classifier.features import FoldedKimExtractor
from lc_classifier.features import HarmonicsExtractor, IQRExtractor, SupernovaeDetectionFeatureExtractor
from lc_classifier.features import PowerRateExtractor
from lc_classifier.features import TurboFatsFeatureExtractor

from lc_classifier.features import FeatureExtractorComposer

In [85]:
'''
2.- Los extractores de características recomendados son:

SupernovaeDetectionFeatureExtractor(band_names),
SNParametricModelExtractor(band_names),
IQRExtractor(band_names),
MHPSExtractor(band_names),
TurboFatsFeatureExtractor(band_names),
GPDRWExtractor(band_names),
PowerRateExtractor(band_names),
PeriodExtractor(band_names),
HarmonicsExtractor(band_names),

FeatureExtractorComposer(
    [
        MHPSExtractor(bands),
        PeriodExtractor(bands),
        GPDRWExtractor(bands),
        FoldedKimExtractor(bands),
        HarmonicsExtractor(bands),
        IQRExtractor(bands),
        PowerRateExtractor(bands),
        TurboFatsFeatureExtractor(bands)
    ]
)
'''
bands = [1,2]
feature_extractor = FeatureExtractorComposer(
    [
        SupernovaeDetectionFeatureExtractor(bands),
        MHPSExtractor(bands),
        GPDRWExtractor(bands),
        FoldedKimExtractor(bands),
        HarmonicsExtractor(bands),
        PowerRateExtractor(bands),
        TurboFatsFeatureExtractor(bands)
    ]
)


In [86]:
url_det='https://raw.githubusercontent.com/joaquinzepeda/Datos/main/datos%20supernovas/detections_SNe_v7.0.1.csv'
url_dfcross='https://raw.githubusercontent.com/joaquinzepeda/Datos/main/datos%20supernovas/dfcrossmatches_prioritized_v7.0.1.csv'
detections = pd.read_csv(url_det, index_col=None)
dfcrossmatches = pd.read_csv(url_dfcross, index_col=None)

In [87]:
from scipy import stats

Conjunto = pd.merge(detections, dfcrossmatches, left_on='objectId', right_on='oid')
slice_cjto = Conjunto[['oid','fid', 'isdiffpos', 'sigmapsf','sigmapsf_corr', 'field',
       'fwhm', 'dec_x', 'magpsf', 'magpsf_corr', 'rcid', 'ra_x', 'sky', 'rb', 'ssmagnr',
       'distpsnr3', 'sgscore2', 'maggaiabright', 'distpsnr2', 'distpsnr1',
       'maggaia', 'exptime', 'drb', 'sgscore3', 'neargaia', 'sgscore1', 'mjd', 'has_stamp', 'ra_y', 'dec_y', 'classALeRCE']]
clean_data = slice_cjto[((np.abs(stats.zscore(slice_cjto['magpsf']))<3)&(slice_cjto['sigmapsf']<1))]

n = len(clean_data)
copy_cleandata = clean_data.copy()


In [88]:
j=0
indice_magpsf_corr = 0
indice_sigmapsf_corr = 0
for columna in copy_cleandata.columns:
    if columna=='magpsf_corr':
        print(columna)
        indice_magpsf_corr = j
    if columna=='sigmapsf_corr':
        print(columna)
        indice_sigmapsf_corr = j
    j+=1
indice_magpsf_corr,indice_sigmapsf_corr 

sigmapsf_corr
magpsf_corr


(9, 4)

In [89]:
for i in range(0,n):
  if pd.isna(clean_data.iloc[i,indice_magpsf_corr]) == True:
    copy_cleandata.iloc[i,indice_magpsf_corr] = copy_cleandata.iloc[i,indice_magpsf_corr-1]
  else:
    pass
for i in range(0,n):
  if pd.isna(clean_data.iloc[i,indice_sigmapsf_corr ]) == True:
    copy_cleandata.iloc[i,indice_sigmapsf_corr ] = copy_cleandata.iloc[i,indice_sigmapsf_corr-1]
  else:
    pass


In [90]:
copy_cleandata.head()

,oid,fid,isdiffpos,sigmapsf,sigmapsf_corr,field,fwhm,dec_x,magpsf,magpsf_corr,...,exptime,drb,sgscore3,neargaia,sgscore1,mjd,has_stamp,ra_y,dec_y,classALeRCE
2,ZTF18aahvndq,1,1.0,0.032161,0.032161,626,1.98,27.008647,16.072975,16.072975,...,30.0,0.999939,0.5,78.111310,0.175976,59004.192674,True,198.722653,27.008637,SNIa
3,ZTF18aahvndq,1,1.0,0.049931,0.049931,626,2.75,27.008624,17.708502,17.708502,...,30.0,0.999956,0.5,78.077545,0.175976,59021.227604,True,198.722653,27.008637,SNIa
6,ZTF18aahvndq,1,1.0,0.044748,0.044748,626,3.49,27.008622,16.626165,16.626165,...,30.0,0.999150,0.5,78.058230,0.175976,59009.258669,True,198.722653,27.008637,SNIa
8,ZTF18aahvndq,1,1.0,0.078094,0.078094,626,2.24,27.008641,18.192377,18.192377,...,30.0,0.999988,0.5,78.038506,0.175976,59036.198090,True,198.722653,27.008637,SNIa
9,ZTF18aahvndq,1,1.0,0.083104,0.083104,626,2.28,27.008652,18.481302,18.481302,...,30.0,0.999997,0.5,78.075700,0.175976,59045.238090,False,198.722653,27.008637,SNIa


In [94]:
# Ahora, tomaremos un ejemplo de cada supernova y graficaremos su curva de luz.
datos_final = copy_cleandata #Las muestras con los NaN reemplazados y sin outliers

In [95]:
#copy_cleandata.columns

In [96]:
try:
    datos_final = datos_final.set_index('oid')
except:
    print('oid is already the index')
datos_final.head()

,fid,isdiffpos,sigmapsf,sigmapsf_corr,field,fwhm,dec_x,magpsf,magpsf_corr,rcid,...,exptime,drb,sgscore3,neargaia,sgscore1,mjd,has_stamp,ra_y,dec_y,classALeRCE
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18aahvndq,1,1.0,0.032161,0.032161,626,1.98,27.008647,16.072975,16.072975,34,...,30.0,0.999939,0.5,78.111310,0.175976,59004.192674,True,198.722653,27.008637,SNIa
ZTF18aahvndq,1,1.0,0.049931,0.049931,626,2.75,27.008624,17.708502,17.708502,34,...,30.0,0.999956,0.5,78.077545,0.175976,59021.227604,True,198.722653,27.008637,SNIa
ZTF18aahvndq,1,1.0,0.044748,0.044748,626,3.49,27.008622,16.626165,16.626165,34,...,30.0,0.999150,0.5,78.058230,0.175976,59009.258669,True,198.722653,27.008637,SNIa
ZTF18aahvndq,1,1.0,0.078094,0.078094,626,2.24,27.008641,18.192377,18.192377,34,...,30.0,0.999988,0.5,78.038506,0.175976,59036.198090,True,198.722653,27.008637,SNIa
ZTF18aahvndq,1,1.0,0.083104,0.083104,626,2.28,27.008652,18.481302,18.481302,34,...,30.0,0.999997,0.5,78.075700,0.175976,59045.238090,False,198.722653,27.008637,SNIa


In [99]:
import warnings
import time
warnings.filterwarnings("ignore")
inicio = time.time()

detections_df= datos_final.rename(columns={'magpsf':'magnitude','mjd':'time', 'fid':'band','sigmapsf':'error'})
features = feature_extractor.compute_features(detections_df)
fin = time.time()
features

ERROR:root:TypeError exception in PeriodExtractor: oid ZTF19aawgopm
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF19acihlft
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF20aamuqwn
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF20aavpnlv
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF20aaynrrh
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF19aawgopm
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF19acihlft
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF20aamuqwn
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF20aavpnlv
'NoneType' object is not iterable
ERROR:root:TypeError exception in PeriodExtractor: oid ZTF20aaynrrh
'NoneType' obj

823.7880561351776


,delta_mag_fid_1,delta_mjd_fid_1,first_mag_1,mean_mag_1,min_mag_1,n_det_1,n_neg_1,n_pos_1,positive_fraction_1,delta_mag_fid_2,...,Skew_2,SmallKurtosis_2,Std_2,StetsonK_2,Pvar_2,ExcessVar_2,SF_ML_amplitude_2,SF_ML_gamma_2,IAR_phi_2,LinearTrend_2
oid,,,,,,,,,,,,,,,,,,,,,
ZTF17aadlxmv,2.244800,42.055613,19.879980,19.087656,17.947500,18.0,0.0,18.0,1.0,1.847730,...,0.767211,1.142780,0.466788,0.893531,1.000000,0.000594,1.874733,0.348850,0.838971,0.001989
ZTF18aaaqexr,0.896138,18.870671,19.133900,18.977915,18.447520,9.0,0.0,9.0,1.0,0.191736,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF18aacdbzx,0.495977,19.991725,19.741877,19.620142,19.432064,14.0,0.0,14.0,1.0,0.786995,...,1.301440,2.497958,0.212669,0.743787,0.999385,0.000068,6.512830,0.948480,0.924340,0.017806
ZTF18aadmssd,3.293953,59.951100,16.887047,18.581230,16.887047,16.0,0.0,16.0,1.0,2.170098,...,-0.066124,-0.621467,0.662524,0.917680,1.000000,0.001411,8.845398,0.926170,0.988949,0.027800
ZTF18aadzfso,1.371363,32.999224,19.840900,19.273643,18.476837,24.0,0.0,24.0,1.0,0.797367,...,-0.444814,-0.743503,0.238186,0.886096,1.000000,0.000121,0.144780,-0.214761,0.899290,0.008069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20abgbxfm,2.981503,68.950197,18.893476,19.555296,18.296597,58.0,0.0,58.0,1.0,2.216758,...,0.100401,-1.245222,0.664534,0.839586,1.000000,0.001109,11.378320,0.987949,0.991831,0.034059
ZTF20abgdtmv,2.669654,33.020729,18.728144,20.046770,18.663546,26.0,0.0,26.0,1.0,1.719614,...,0.070929,-0.974009,0.528035,0.858666,1.000000,0.000697,7.053065,0.740276,0.979279,0.039661
ZTF20abgfekk,1.627867,31.928044,19.095242,17.742374,17.467375,32.0,0.0,32.0,1.0,1.534458,...,2.443501,6.694064,0.364999,0.679766,1.000000,0.000406,15.000000,1.116547,0.952584,-0.026970


In [106]:
tiempo_de_extraccion = (fin-inicio)/60
print(f'La extracción de caracteristicas se demoro {"{:.2f}".format(tiempo_de_extraccion)} minutos')

La extracción de caracteristicas se demoro 13.73 minutos


In [ ]:
features.reset_index()

In [108]:
#-------Por revisar------------
'''
oid ZTF19aawgopm
oid ZTF19acihlft
oid ZTF20aamuqwn
oid ZTF20aavpnlv
oid ZTF20aaynrrh
oid ZTF19aawgopm
oid ZTF19acihlft
'''
datos_final[datos_final.index=='ZTF19acihlft']

,fid,isdiffpos,sigmapsf,sigmapsf_corr,field,fwhm,dec_x,magpsf,magpsf_corr,rcid,...,exptime,drb,sgscore3,neargaia,sgscore1,mjd,has_stamp,ra_y,dec_y,classALeRCE
oid,,,,,,,,,,,,,,,,,,,,,
ZTF19acihlft,1,1.0,0.162876,100.000000,390,2.71,-10.183966,19.108986,17.352884,16,...,30.0,0.999879,0.5,55.420258,0.164857,58782.211250,True,325.972675,-10.183955,SNIa
ZTF19acihlft,1,1.0,0.104800,0.030519,390,5.07,-10.183993,17.668047,16.877300,16,...,30.0,0.999487,0.5,55.518864,0.164857,58830.108183,False,325.972675,-10.183955,SNIa
ZTF19acihlft,1,1.0,0.029123,0.015243,390,2.81,-10.183949,16.201582,15.935588,16,...,30.0,0.999991,0.5,55.441986,0.164857,58789.182153,True,325.972675,-10.183955,SNIa
ZTF19acihlft,2,1.0,0.153177,100.000000,390,1.65,-10.183935,19.155579,16.822386,16,...,30.0,0.999755,0.5,55.316890,0.164857,58782.181609,True,325.972675,-10.183955,SNIa
ZTF19acihlft,2,1.0,0.038713,100.000000,390,2.47,-10.183946,16.194202,15.757371,16,...,30.0,0.999999,0.5,55.405365,0.164857,58812.089965,True,325.972675,-10.183955,SNIa
ZTF19acihlft,2,1.0,0.030518,100.000000,390,2.51,-10.183954,16.297045,15.825058,16,...,30.0,0.999999,0.5,55.453045,0.164857,58789.211586,True,325.972675,-10.183955,SNIa
ZTF19acihlft,2,1.0,0.059134,100.000000,390,2.51,-10.183957,17.579530,16.471671,16,...,30.0,0.999996,0.5,55.440636,0.164857,58785.233032,True,325.972675,-10.183955,SNIa


In [149]:
cells = features[features.index=='ZTF18aaaqexr'].values
indices=[]
i=0
for cell in cells[0]:
    if str(cell)=='nan':
        indices.append(i)
    i+=1    
print(indices)
# Como se puede observar, hay algunas caracteristicas que contienen nan, estas hay que revisarlas
# por separado pues se deben a errores
nan = features.iloc[:, indices]
nan.head()

[23, 24, 25, 26, 27, 34, 35, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121]


,MHPS_ratio_2,MHPS_low_2,MHPS_high_2,MHPS_non_zero_2,MHPS_PN_flag_2,Psi_CS_2,Psi_eta_2,Amplitude_2,AndersonDarling_2,Autocor_length_2,...,Skew_2,SmallKurtosis_2,Std_2,StetsonK_2,Pvar_2,ExcessVar_2,SF_ML_amplitude_2,SF_ML_gamma_2,IAR_phi_2,LinearTrend_2
oid,,,,,,,,,,,,,,,,,,,,,
ZTF17aadlxmv,26.178661,1.065202,0.040690,36.0,0.0,0.361719,0.345065,0.923865,0.999412,3.0,...,0.767211,1.142780,0.466788,0.893531,1.000000,0.000594,1.874733,0.348850,0.838971,0.001989
ZTF18aaaqexr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF18aacdbzx,8.221117,0.073785,0.008975,20.0,0.0,0.364474,0.463809,0.393497,0.999341,2.0,...,1.301440,2.497958,0.212669,0.743787,0.999385,0.000068,6.512830,0.948480,0.924340,0.017806
ZTF18aadmssd,342.520355,12.741676,0.037200,30.0,0.0,0.270602,0.644786,1.085049,0.990112,6.0,...,-0.066124,-0.621467,0.662524,0.917680,1.000000,0.001411,8.845398,0.926170,0.988949,0.027800
ZTF18aadzfso,2.862207,0.223176,0.077974,30.0,0.0,0.412181,0.379704,0.398684,0.995344,5.0,...,-0.444814,-0.743503,0.238186,0.886096,1.000000,0.000121,0.144780,-0.214761,0.899290,0.008069


In [169]:
banned_features = [
   'W1', 'W2', 'W3', 'W4',
   'iqr_1',
   'iqr_2',
   'delta_mjd_fid_1',
   'delta_mjd_fid_2',
   'last_mjd_before_fid_1',
   'last_mjd_before_fid_2',
   'g-r_ml',
   'MHAOV_Period_1', 'MHAOV_Period_2'
]
banned_features1 = ['mean_mag_1',
   'mean_mag_2',
   'min_mag_1',
   'min_mag_2',
   'Mean_1',
   'Mean_2',
   'n_det_1',
   'n_det_2',
   'n_pos_1',
   'n_pos_2',
   'n_neg_1',
   'n_neg_2',
   'first_mag_1',
   'first_mag_2',
   'MHPS_non_zero_1',
   'MHPS_non_zero_2',
   'MHPS_PN_flag_1',
   'MHPS_PN_flag_2']


features_clean = features.drop(['Eta_e_1','MaxSlope_1','Eta_e_2','MaxSlope_2'], axis=1)
features_clean = features_clean.drop(banned_features1, axis=1)
features_clean

,delta_mag_fid_1,delta_mjd_fid_1,positive_fraction_1,delta_mag_fid_2,delta_mjd_fid_2,positive_fraction_2,MHPS_ratio_1,MHPS_low_1,MHPS_high_1,MHPS_ratio_2,...,Skew_2,SmallKurtosis_2,Std_2,StetsonK_2,Pvar_2,ExcessVar_2,SF_ML_amplitude_2,SF_ML_gamma_2,IAR_phi_2,LinearTrend_2
oid,,,,,,,,,,,,,,,,,,,,,
ZTF17aadlxmv,2.244800,42.055613,1.0,1.847730,48.854525,1.0,24.510580,3.971278,0.162023,26.178661,...,0.767211,1.142780,0.466788,0.893531,1.000000,0.000594,1.874733,0.348850,0.838971,0.001989
ZTF18aaaqexr,0.896138,18.870671,1.0,0.191736,10.058981,1.0,0.216739,0.046887,0.216330,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF18aacdbzx,0.495977,19.991725,1.0,0.786995,27.982234,1.0,0.752135,0.016245,0.021599,8.221117,...,1.301440,2.497958,0.212669,0.743787,0.999385,0.000068,6.512830,0.948480,0.924340,0.017806
ZTF18aadmssd,3.293953,59.951100,1.0,2.170098,76.909931,1.0,337.794556,34.621597,0.102493,342.520355,...,-0.066124,-0.621467,0.662524,0.917680,1.000000,0.001411,8.845398,0.926170,0.988949,0.027800
ZTF18aadzfso,1.371363,32.999224,1.0,0.797367,36.979248,1.0,0.751511,0.199288,0.265183,2.862207,...,-0.444814,-0.743503,0.238186,0.886096,1.000000,0.000121,0.144780,-0.214761,0.899290,0.008069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20abgbxfm,2.981503,68.950197,1.0,2.216758,69.925914,1.0,191.005722,13.320155,0.069737,221.095032,...,0.100401,-1.245222,0.664534,0.839586,1.000000,0.001109,11.378320,0.987949,0.991831,0.034059
ZTF20abgdtmv,2.669654,33.020729,1.0,1.719614,41.949225,1.0,112.711197,14.528273,0.128898,88.620247,...,0.070929,-0.974009,0.528035,0.858666,1.000000,0.000697,7.053065,0.740276,0.979279,0.039661
ZTF20abgfekk,1.627867,31.928044,1.0,1.534458,31.981505,1.0,3.911379,0.107875,0.027580,11.340262,...,2.443501,6.694064,0.364999,0.679766,1.000000,0.000406,15.000000,1.116547,0.952584,-0.026970


In [170]:
#features_clean.to_csv('features_clean.csv')

In [171]:
print('NaN occurrences in DataFrame:')
print(features_clean.isnull().sum().sum())

NaN occurrences in DataFrame:
12417


# Separación del conjunto en ENTRENAMIENTO y TEST: 
Dividiremos el conjunto en 70% para entrenamiento y 30% para test

In [145]:

from sklearn.utils import shuffle

features_clean.reset_index()
#--------------------le agregamos la clase que le corresponde a cada registro---------------------------
df_con_target = pd.merge(features_clean, dfcrossmatches[['oid','classALeRCE']], left_on='oid', right_on='oid')

#--------------------Mezclamos los datos de tal manera de poder tomar muestras representativas del conjunto-----------------
df_shuffle = shuffle(df_con_target)


NaN occurrences in DataFrame:
19814


In [128]:
df_shuffle.head()

,oid,delta_mag_fid_1,delta_mjd_fid_1,positive_fraction_1,delta_mag_fid_2,delta_mjd_fid_2,positive_fraction_2,MHPS_ratio_1,MHPS_low_1,MHPS_high_1,...,SmallKurtosis_2,Std_2,StetsonK_2,Pvar_2,ExcessVar_2,SF_ML_amplitude_2,SF_ML_gamma_2,IAR_phi_2,LinearTrend_2,classALeRCE
1048,ZTF19abgmkef,2.134502,57.003368,1.0,1.213207,69.829931,1.0,196.159134,7.519462,0.038333,...,-0.623996,0.317441,0.882110,1.0,0.000223,3.663800,0.885446,0.979227,0.013324,SNII
911,ZTF19aaxffum,2.725270,54.918426,1.0,2.026674,61.904016,1.0,122.327797,10.734665,0.087753,...,0.013289,0.538886,0.851194,1.0,0.000835,3.897010,0.589056,0.957731,0.014419,SNIa
366,ZTF18acpeesd,1.694427,24.978206,1.0,1.671331,52.929144,1.0,30203.408203,2.715898,0.000090,...,-0.675125,0.598654,0.938816,1.0,0.000894,11.855870,0.986467,0.984640,0.034354,SNIa
789,ZTF19aatesgp,2.716905,52.016400,1.0,2.976043,66.991736,1.0,279.501190,27.022142,0.096680,...,-0.193373,0.828920,0.850590,1.0,0.002220,15.000000,0.995879,0.990785,0.046203,SNIbc
1981,ZTF20abasxmb,2.819666,42.021644,1.0,1.246100,37.933194,1.0,16.353119,2.397396,0.146602,...,-1.162153,0.448851,0.934096,1.0,0.000720,4.794406,0.660739,0.924082,0.023197,SNIa


In [173]:
rows= df_shuffle.shape[0] 

# ENTRENAMIENTO DESDE 0 A int(rows*0.7)
train = df_shuffle.iloc[0:int(rows*0.7)]

# TEST DESDE int(rows*0.7) HASTA EL FINAL
test = df_shuffle.iloc[int(rows*0.7):rows]

# SE EXTRAEN LOS TARGETS PARA ENTRENAR
target_entrenamiento = np.array(train['classALeRCE'])

train_sin_target = train.drop(['oid','classALeRCE'], axis=1)
data = train_sin_target.to_numpy(dtype='float32')
data

array([[3.0459800e+00, 6.2877918e+01, 1.0000000e+00, ..., 1.7937380e-01,
        9.7045815e-01, 1.8331505e-02],
       [3.2698750e+00, 1.0088787e+02, 1.0000000e+00, ..., 8.7784058e-01,
        9.9137539e-01, 2.9921709e-02],
       [3.5642409e+00, 1.1877930e+02, 1.0000000e+00, ..., 4.4697616e-01,
        9.6357232e-01, 1.5884627e-02],
       ...,
       [1.9043580e+00, 2.5895302e+01, 1.0000000e+00, ..., 6.7037904e-01,
        9.3798959e-01, 2.4506576e-02],
       [1.0224650e+00, 4.3922421e+01, 1.0000000e+00, ...,           nan,
                  nan,           nan],
       [2.3725801e+00, 3.5981991e+01, 1.0000000e+00, ..., 9.6583247e-02,
        9.1783321e-01, 1.7704556e-02]], dtype=float32)